In [1]:
import cv2
import mediapipe as mp
import urllib.request as urlreq
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Ellipse, Rectangle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob
import pandas as pd

#from rembg import remove

from config import *
from cropping import *
from landmarks_utils import *
from preprocessing import *
from face_landmarking_model import *
from training import *
from face_dataset import FaceDataset, EnsembleSampler
import itertools

In [2]:
# x_inp, y_inp, path_list, angles = prepare_training_landmarks()

In [3]:
# save_preprocessed_data(x_inp, y_inp, path_list, angles)

In [4]:
# Prepare best groups for ensemble projection
groups = [os.path.basename(os.path.normpath(path_string)) for path_string in glob("./AI_Morphometrics/*/", recursive = False)]
groups_results = pd.read_json('training_results_means.json', orient='index')
groups_results.columns = ['result']
list_df = pd.read_csv('./preprocessed_data/path_list.txt', names=['text'], header=None)
counts = {}
for group in groups:
    counts[group] = len(list_df[list_df['text'].str.contains(group)])
compare = pd.concat([groups_results, pd.DataFrame(counts.items(), columns=['group', 'sample']).set_index('group')], axis = 1)
#plt.scatter(x=compare['sample'], y=compare['result'])
num_selected_groups = 10
best_groups = compare.dropna().sort_values(by=['sample'])[-num_selected_groups:].sort_values(by=['result']).index.to_list()

In [5]:
results = {}
valid_results = {}
cached_models = {}
times = {}
projectors = [2]
rotations = [True]
learning_rates = [0.0005, 0.001]
num_parents = [5]


hyperparameter_combinations = list(itertools.product(projectors, rotations, learning_rates, num_parents))

for num_projectors, rotate, lr, num_parent_landmarks in tqdm(hyperparameter_combinations):
    combination = f"{num_projectors}_{rotate}_{lr}_{num_parent_landmarks}"
    model, optimizers, schedulers, datasets, dataloaders = prepare_trainers(best_groups, 
                                                                            num_parent_landmarks = num_parent_landmarks, 
                                                                            projectors=num_projectors, 
                                                                            rotate=rotate, 
                                                                            lr_projection=0.01,
                                                                            lr_cnn=0.08,
                                                                            lr_ffn=lr
                                                                            )
    cache, main_cache, time_cache = train(model,optimizers, 
                              schedulers, 
                              datasets, 
                              dataloaders, 
                              pretrain_epochs=150,
                              cnn_epochs=10,
                              ffn_epochs=10
                              )
    
    results[combination] = cache
    valid_results[combination] = main_cache
    cached_models[combination] = model
    times[combination] = time_cache
            

  0%|          | 0/2 [00:00<?, ?it/s]

Pre-training epoch: 149, loss: 0.0005247946828603745..
 Freezing raw_projection, training CNN_focusing.
Post-training epoch: 159, final-loss: 0.009104503318667412, raw-loss: 0.00010404462955193594.
 Freezing CNN_focusing, training top FFN.


 50%|█████     | 1/2 [40:32<40:32, 2432.24s/it]

Pre-training epoch: 149, loss: 0.0001779945014277473..331413, raw-loss: 0.00010330187797080725.
 Freezing raw_projection, training CNN_focusing.


 50%|█████     | 1/2 [56:30<56:30, 3390.18s/it]


KeyboardInterrupt: 

In [12]:
import plotly.graph_objs as go
import plotly.express as px

fig = go.Figure()

for combination, losses in results.items():
    scatter = go.Scatter(
        x=list(range(len(losses)))[50:],
        y=losses[50:],
        mode='lines',
        name=combination,
        hovertext=combination
        )
    
    fig.add_trace(scatter)

fig.show()

In [13]:

fig = go.Figure()

for combination, losses in valid_results.items():
    scatter = go.Scatter(
        x=list(range(len(losses))),
        y=losses,
        mode='lines',
        name=combination,
        hovertext=combination
        )
    
    fig.add_trace(scatter)

fig.show()

In [ ]:
model.train_phase = 1

In [1]:
dataset = datasets["main"]
dataset.pretraining = False
idx = torch.randint(dataset.__len__(),(1,))
print(idx)
img_path = dataset.path_list[idx]
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
landmarks, inputs_pathway, image_detail = model.predict(image, face_detail=True)
landmarks = landmarks.reshape(-1,2)
display_landmarks(landmarks, image_detail, pixel_scale=False, origin='upper_left')

# display_landmarks(inputs_pathway.reshape(-1,2), image_detail, pixel_scale=False, origin='upper_left')

NameError: name 'datasets' is not defined

In [ ]:
@torch.no_grad()
def show_some_results(group = None, idx = None):
    if group is None:
        group = sample_groups[np.random.randint(len(sample_groups))]
    
    for file in os.listdir(group):
        if '.TPS' in file or '.tps' in file:
             tps = readtps(group + '/' + file, group)
        
    if idx is None:
        idx = np.random.randint(len(tps['im']))
        
    true_landmarks = tps['coords'][:, :, idx]
    img_path = group + '/' + tps['im'][idx]
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    face_detail = crop_face_only(image)
    if face_detail:
        subimage, xmin, ymin, xmax, ymax = face_detail
    
    # Both model use float(0,1) for x and y axis
    input_landmarks = MediaPipe_model(subimage)
    #input_landmarks = np.concatenate((LBF_model(image), MediaPipe_model(image)), axis = 0)

    #true_landmarks = np.subtract(true_landmarks, (xmin, image.shape[0] - ymax))
    #true_landmarks = np.divide(true_landmarks, (subimage.shape[1], subimage.shape[0]))
     
    # Převod původních pixelových na float, flip y-axis
    true_landmarks = torch.from_numpy(1 - np.divide(true_landmarks, (image.shape[1], image.shape[0]))).to(device)

    # batch_dim = 0
    input_landmarks = torch.from_numpy(input_landmarks.reshape(1,-1)).float().to(device)
    raw_landmarks, _ = model.raw_projection(input_landmarks, None)

    multicrop = make_landmark_crops(raw_landmarks, subimage)
    
    #předělat a použít prediction method
    projection, _, _ = model(input_landmarks, true_landmarks.reshape(1,-1), multicrop = multicrop[None,:,:,:], image_shape = subimage.shape)
    projection = projection.cpu().detach().numpy().reshape(true_landmarks.shape)
    
    # tady připočíst xmin a ten druhý pól image_shape - ymax
    return projection, image, true_landmarks, raw_landmarks